In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Read keyword from excel and set the target URL

In [2]:
#Target link
url = "https://www.baby-kingdom.com/search.php?mod=forum"

#read keyword list from excel file
product = pd.read_excel("Banks.xlsx")

#drop null row
keyword_list = product["Bank"].dropna()
keyword_list = keyword_list.values.tolist()
print(keyword_list)

['中國銀行', '交通銀行', '東亞銀行', '中信銀行', '中國建設銀行', '集友銀行', '創興銀行', '花旗銀行', '大新銀行', '星展銀行', '富邦銀行', '恒生銀行', '香港上海滙豐銀行', '中國工商銀行', '南洋商業銀行', '華僑永亨銀行', '大眾銀行', '上海商業銀行', '渣打銀行', '大生銀行', '大有銀行', '招商永隆銀行', '中銀', '交通銀行', '東亞', '中信', '中建', '集友', '創興', '花旗', '大新', '星展', '富邦', '恒生', '滙豐', '大眾', '上商業', '渣打', '大生', '大有', '招商', 'HSBC', 'Hang Seng', 'BOC']


#Setup the chrome drive

In [3]:
#setup engine for web scraping
options = webdriver.ChromeOptions()

#path of the chrome driver 
chromedriver_path='chromedriver.exe'

#open the chrome driver
driver = webdriver.Chrome(options=options, executable_path=chromedriver_path) 

C:\Users\User\AppData\Local\Temp/ipykernel_23964/1609432386.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=chromedriver_path)


#Create dataframe to stores the data

In [4]:
#extract comments 
def extractData(soup,keyword):

    for i in soup.find_all("li",{"class":"pbw"}):
        title = i.find("h3",{"class":"xs3"}).text
        text = i.find_all("p")[1].text
        numComment = i.find_all("p")[0].text.split('-')[0].split(' ')[0]
        numView = i.find_all("p")[0].text.split('-')[1].split(' ')[1]
        time = i.find_all("span")[0].text.split(' ')
        date = time[0]
        create_time = time[1]
        author = i.find_all("span")[1].text.replace("\n","")
        topic = i.find_all("span")[2].text
        link = i.find_all('a', href=True)[0].get("href")

        df_post.loc[len(df_post)]= [keyword, link, date, create_time, topic, title, author, numComment, numView, text]
   
    return 

In [5]:

df_post = pd.DataFrame(columns=["Keyword","Link", "Date", "Create Time", "Theme", "Topic", "Author", "Number of comment", "Number of Veiw", "Text"])

for arc_key in keyword_list:
    driver.get("https://www.baby-kingdom.com/search.php?mod=forum")
    inputElement = driver.find_element(By.ID,"srchtxt")
    inputElement.send_keys(arc_key)
    inputElement.submit()
    window_after = driver.window_handles[0]
    driver.switch_to.window(window_after)
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #extract html for further processing
    #temp_keyword, temp_link, temp_time, temp_topic, temp_title, temp_author, temp_numComment, temp_numView, temp_text = extractData(soup,arc_key)
    #extractData(soup,arc_key)

    soup = BeautifulSoup(html)
    while soup.find("i",{"class":"ico_on icon-pagin-generic_nextPageOn"}) != None:
        #print("next")
        if soup.find('i',{"class":"ico_off icon-pagin-generic_nextPageOn"}) != None:
            break 
        nextpage = driver.find_element(By.CLASS_NAME,"icon-pagin-generic_nextPageOn")
        nextpage.click()
        window_after = driver.window_handles[0]
        driver.switch_to.window(window_after)
        html = driver.page_source
        soup =BeautifulSoup(html)

        extractData(soup,arc_key)
        
    #Test

        
#Close the driver...
driver.quit() 
print(f"End of scraping {len(df_post)} has been scraped...")      

End of scraping 336 has been scraped...


In [6]:
df_post

,Keyword,Link,Date,Create Time,Theme,Topic,Author,Number of comment,Number of Veiw,Text
0,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-1-16,10:45,二手市場,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,Flora3k,4,719,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封若有興趣可PM我。
1,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-7-9,16:37,家庭理財,建設銀行 中國銀行,Wishsss,57,10923,比較之下，點解建設銀行個股價咁硬淨?\n建設銀行派完息都仲升
2,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-6-29,00:44,自由報料,"申請政府一萬元, 中國銀行及滙豐有什麽優惠？",mimikins8888,11,2133,申請政府一萬元，中國銀行及滙豐有什麽優惠？
3,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-5-22,13:52,自由講場,中國銀行取號排隊,bkpokk,5,3012,"好耐無去銀行,\n請問 中國銀行 有冇網上取號排隊 ？"
4,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-3-8,13:02,家庭理財,MPF－中國銀行,DFSON,7,1907,請問邊隻基金表現比較好？
...,...,...,...,...,...,...,...,...,...,...
331,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-12-29,16:55,自由講場,知唔知的人係中國銀行排隊做乜?,2mings,2,567,
332,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-12-5,22:50,家庭理財,中國銀行有話十二月上市?,chrischeung,1,660,
333,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-10-1,15:09,自由講場,請問用係提款機恒生戶口可唔可以轉帳去中國銀行呢?,xp,14,415,
334,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-8-6,11:38,家庭理財,中國銀行 & 盈富基金 & 地鐵,wapwap,6,1821,


In [11]:
#create UID for post
uid = []
for i in range(len(df_post)):
    uid.append("BK-"+str(i).zfill(8))

df_post.insert(loc = 0, column = 'Post_UID', value = uid)
                   
#output data
df_post.to_excel('BabyKingdom_Bank_Data.xlsx',encoding= 'utf-8-sig',index = None) 


C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [12]:
df_post

,Post_UID,Keyword,Link,Date,Create Time,Theme,Topic,Author,Number of comment,Number of Veiw,Text
0,00000000,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-1-16,10:45,二手市場,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,Flora3k,4,719,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封若有興趣可PM我。
1,00000001,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-7-9,16:37,家庭理財,建設銀行 中國銀行,Wishsss,57,10923,比較之下，點解建設銀行個股價咁硬淨?\n建設銀行派完息都仲升
2,00000002,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-6-29,00:44,自由報料,"申請政府一萬元, 中國銀行及滙豐有什麽優惠？",mimikins8888,11,2133,申請政府一萬元，中國銀行及滙豐有什麽優惠？
3,00000003,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-5-22,13:52,自由講場,中國銀行取號排隊,bkpokk,5,3012,"好耐無去銀行,\n請問 中國銀行 有冇網上取號排隊 ？"
4,00000004,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,20-3-8,13:02,家庭理財,MPF－中國銀行,DFSON,7,1907,請問邊隻基金表現比較好？
...,...,...,...,...,...,...,...,...,...,...,...
331,00000331,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-12-29,16:55,自由講場,知唔知的人係中國銀行排隊做乜?,2mings,2,567,
332,00000332,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-12-5,22:50,家庭理財,中國銀行有話十二月上市?,chrischeung,1,660,
333,00000333,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-10-1,15:09,自由講場,請問用係提款機恒生戶口可唔可以轉帳去中國銀行呢?,xp,14,415,
334,00000334,中國銀行,https://www.baby-kingdom.com/forum.php?mod=vie...,05-8-6,11:38,家庭理財,中國銀行 & 盈富基金 & 地鐵,wapwap,6,1821,
